In [1]:
# Reminders: 
# Make sure on the top right hand corner, the kernel selected is conda_mxnet_p36
# A kernel is the computational engine that runs the code. 
# Selecting the wrong kernel can potentially cause the it to not understand the code and excute incorrectly.
# Imagine setting the language setting of a machine translator to English but feeding it Spanish input 
# The translator wouldn't function properly, would it?

In [3]:
# Install some code libraries
!python3 -m pip install -qU pip
!python3 -m pip install -qU setuptools wheel
!python3 -m pip install -qU "mxnet<2.0.0"
!python3 -m pip install -qU autogluon


In [7]:
# Importing the newly installed AutoGluon code library
from autogluon.tabular import TabularPredictor, TabularDataset
import autogluon.core as ag

# Load coding libraries
from sklearn.model_selection import train_test_split
import pandas as pd

In [5]:
#Load the csv dataset in as a autogluon dataset object 
df_train = TabularDataset(data="./normalized.csv")

# Preview the first five rows to make sure it's loaded correctly
df_train.head()



,genre,meanTempogram,stdTempogram,varTempogram,meanMFCC_1,stdMFCC_1,varMFCC_1,meanMFCC_2,stdMFCC_2,varMFCC_2,...,varSpectralRollOff,meanSpectralBandwidth,stdSpectralBandwidth,varSpectralBandwidth,meanSpectralContrast,stdSpectralContrast,varSpectralContrast,meanSpectralFlatness,stdSpectralFlatness,varSpectralFlatness
0,reggae,0.195446,0.163096,0.095401,0.726771,0.419814,0.211750,0.091864,0.370346,0.198718,...,0.215297,0.847715,0.374602,0.192241,0.286421,0.122342,0.062079,0.296769,0.363137,0.132012
1,reggae,0.251674,0.209781,0.127566,0.601865,0.411215,0.204393,0.617099,0.448078,0.266062,...,0.311284,0.407825,0.488073,0.293583,0.652288,0.562604,0.424493,0.009669,0.029061,0.000862
2,reggae,0.309029,0.243334,0.152018,0.785708,0.285397,0.111182,0.320733,0.457891,0.275196,...,0.443686,0.901846,0.500665,0.306064,0.190606,0.062760,0.029747,0.145615,0.257409,0.066378
3,reggae,0.236639,0.177434,0.105049,0.605709,0.372250,0.172635,0.606323,0.299988,0.145431,...,0.332173,0.431983,0.486804,0.292338,0.632024,0.605462,0.471393,0.011640,0.033886,0.001169
4,reggae,0.174107,0.163321,0.095551,0.752637,0.387832,0.185024,0.348909,0.635720,0.465276,...,0.613481,0.686209,0.584828,0.395828,0.315782,0.124489,0.063318,0.215885,0.452068,0.204520


In [25]:
# Splitting in train and validation datasets
train_data, val_data = train_test_split(
    df_train, test_size=0.2, shuffle=True, random_state=23
)

# Printing the first rows
train_data.head()

train_data = df_train

In [ ]:
# These are some hyperparameters you can configure to train the models. 
# A lot of times it's trial and error before finding the right number!


time_limit = 60*60  # This means setting the longest wait tiem to 3600 seconds (an hour)

nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 10,  # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'layers': ag.space.Categorical([100], [1000], [200, 100], [300, 200, 100]),  # each choice for categorical hyperparameter 'layers' corresponds to list of sizes for each NN layer to use
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN': nn_options,  # NOTE: comment this line out if you get errors on Mac OSX
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

num_trials = 15  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using Bayesian optimization routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

predictor = TabularPredictor(label="genre").fit( train_data=train_data, time_limit=time_limit, presets='best_quality')


# predictor = TabularPredictor(label="genre").fit(
#     train_data=train_data, time_limit=time_limit,
#     hyperparameters=hyperparameters, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs, num_bag_folds = 7, num_bag_sets = 3, num_stack_levels=3)
#presets='best_quality' =   auto = "true"  = choose num_bag_folds = 7, num_stack_levels = 2, num_stack_levels=2 automatically


No path specified. Models will be saved in: "AutogluonModels/ag-20211124_231622/"
Presets specified: ['best_quality']
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20211124_231622/"
AutoGluon Version:  0.3.1
Train Data Rows:    1000
Train Data Columns: 63
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	10 unique label values:  ['reggae', 'hiphop', 'pop', 'rock', 'classical', 'jazz', 'country', 'metal', 'blues', 'disco']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count: 10
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    4692.66 MB
	Train Data (Original)  Memory Usage: 0.5 MB (0.0% of available memory)
	Inferring 

In [31]:
# Run the code below
predictor.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.801,2.764444,458.539536,0.000542,0.513906,2,True,14
1,WeightedEnsemble_L3,0.799,4.264441,1362.396916,0.000511,0.444993,3,True,26
2,NeuralNetFastAI_BAG_L2,0.796,4.209261,1172.389026,0.757616,19.392066,2,True,15
3,LightGBMXT_BAG_L2,0.793,3.506920,1226.606650,0.055274,73.609689,2,True,16
4,LightGBM_BAG_L2,0.791,3.506315,1342.559858,0.054669,189.562897,2,True,17
5,NeuralNetMXNet_BAG_L1,0.790,1.660368,59.206702,1.660368,59.206702,1,True,12
6,CatBoost_BAG_L2,0.789,3.561954,1752.422454,0.110308,599.425493,2,True,20
7,NeuralNetFastAI_BAG_L1,0.788,0.274074,26.688712,0.274074,26.688712,1,True,3
8,XGBoost_BAG_L2,0.782,3.516606,1355.795274,0.064960,202.798313,2,True,23
9,LightGBMLarge_BAG_L2,0.778,3.569392,1967.695393,0.117747,814.698432,2,True,25


/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.8.0` and `torch==1.10.0+cu102` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


ImportError: cannot import name 'AutoGluonTraining'

CompletedProcess(args='./setup.sh', returncode=126)

CompletedProcess(args='apt install unzip', returncode=127)

CompletedProcess(args='docker system prune -af', returncode=0)

CompletedProcess(args='/bin/bash ./build_push_inference.sh 836306170180 us-east-1 autogluon-sagemaker-inference 836306170180.dkr.ecr.us-east-1.amazonaws.com 763104351884 763104351884.dkr.ecr.us-east-1.amazonaws.com/mxnet-inference:1.7.0-cpu-py3', returncode=0)

CompletedProcess(args='docker system prune -af', returncode=0)